# Weights

## Load Graph

In [1]:
import os
import sys
from pathlib import Path
import plotly.graph_objects as go
import plotly.io as pio
from dotenv import load_dotenv

# Add the src directory to the Python path
project_root = Path.cwd().parent.parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

from src.maritime_module.utils.port_utils import Boundaries
from src.maritime_module.utils.plot_utils import PlotlyChart
from src.maritime_module.utils.geometry_utils import Buffer
import geopandas as gpd
from shapely.geometry import shape, LineString, MultiPolygon, Point, Polygon, box


from src.maritime_module.core.graph import H3Graph, Weights
from src.maritime_module.core.s57_data import ENCDataFactory

# Load environment variables from .env file at the project root
load_dotenv(project_root / ".env")
pio.renderers.default = "notebook_connected"

output_dir = Path.cwd() / 'output'
output_dir.mkdir(exist_ok=True)

db_params = {
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT')
}

factory = ENCDataFactory(source=db_params, schema="us_enc_all")
h3 = H3Graph(data_factory=factory,
             route_schema_name="routes",
             graph_schema_name="graph")

2025-10-07 22:31:22,051 - src.maritime_module.core.s57_data - INFO - Source is a dictionary, initializing PostGISManager.
2025-10-07 22:31:22,078 - src.maritime_module.core.s57_data - INFO - Successfully connected to database 'ENC_db'


In [2]:
from src.maritime_module.utils.db_utils import PostGISConnector

pg = PostGISConnector(db_params=db_params, schema="graph")
# pg.drop_columns(schema_name="graph",
#                 table_name="fine_graph_01_opt_edges",
#                 columns=["final_weight", "static_weight_factor"])

In [3]:
graph_name = "fine_graph_01_opt"
# Initialize the Weights manager with the same data factory
weights_manager = Weights(data_factory=factory)
weights_manager.clean_graph_postgis(graph_name)

2025-10-07 22:31:22,167 - src.maritime_module.core.graph - INFO - Weights manager initialized with default S57 classifier
2025-10-07 22:31:22,168 - src.maritime_module.core.graph - INFO - Default static layers from config: 15 layers
2025-10-07 22:31:22,169 - src.maritime_module.core.graph - INFO - === Cleaning PostGIS Graph ===
2025-10-07 22:31:22,169 - src.maritime_module.core.graph - INFO - Table: graph.fine_graph_01_opt_edges
2025-10-07 22:31:22,295 - src.maritime_module.core.graph - INFO - Dropping 14 columns...
2025-10-07 22:31:22,369 - src.maritime_module.core.graph - INFO - === PostGIS Graph Cleaned ===
2025-10-07 22:31:22,369 - src.maritime_module.core.graph - INFO - Dropped 14 columns
2025-10-07 22:31:22,370 - src.maritime_module.core.graph - INFO - Kept 9 columns (including weight, geom)


{'columns_dropped': 14,
 'columns_kept': ['id',
  'source_str',
  'target_str',
  'source_x',
  'source_y',
  'target_x',
  'target_y',
  'weight',
  'geometry'],
 'columns_removed': ['base_weight',
  'adjusted_weight',
  'blocking_factor',
  'penalty_factor',
  'bonus_factor',
  'ukc_meters',
  'ft_hor_clearance',
  'ft_ver_clearance',
  'ft_depth',
  'ft_sounding',
  'ft_sounding_point',
  'wt_static_blocking',
  'wt_static_penalty',
  'wt_static_bonus']}

In [4]:
# Load a pre-existing graph from PostGIS
# This assumes a graph was previously saved with a prefix like 'fine_graph_01'

try:
    G = h3.load_graph_from_postgis(table_prefix=graph_name)
    print(f"Successfully loaded graph with {G.number_of_nodes():,} nodes and {G.number_of_edges():,} edges.")
except Exception as e:
    print(f"Could not load graph. Please ensure a graph has been saved to PostGIS first. Error: {e}")
    G = None

# 1. Get the boundary of the graph itself
nodes_df = gpd.GeoDataFrame(geometry=[Point(n) for n in G.nodes()], crs="EPSG:4326")
graph_boundary = nodes_df.union_all().convex_hull

2025-10-07 22:31:22,391 - src.maritime_module.core.graph - INFO - Loading graph from PostGIS schema 'graph' tables: fine_graph_01_opt_nodes, fine_graph_01_opt_edges
2025-10-07 22:31:40,138 - src.maritime_module.core.graph - INFO - Loaded and processed 356,996 nodes in 17.747s
2025-10-07 22:33:58,283 - src.maritime_module.core.graph - INFO - Loaded and processed 1,416,342 edges in 138.144s
2025-10-07 22:33:58,621 - src.maritime_module.core.graph - INFO - Graph loaded from PostGIS: 356,996 nodes, 1,416,342 edges in 156.061s
2025-10-07 22:33:58,622 - src.maritime_module.core.graph - INFO - === PostGIS Graph Load Operation Performance Summary ===
2025-10-07 22:33:58,623 - src.maritime_module.core.graph - INFO - Timing Metrics:
2025-10-07 22:33:58,623 - src.maritime_module.core.graph - INFO -   nodes_load_time: 2.806s
2025-10-07 22:33:58,624 - src.maritime_module.core.graph - INFO -   nodes_processing_time: 14.940s
2025-10-07 22:33:58,624 - src.maritime_module.core.graph - INFO -   edges_lo

 ## Initialize Weights Class

 ## Apply Static Feature Weights

In [5]:
# 2. Get the list of ENCs intersecting this boundary
enc_list = factory.get_encs_by_boundary(graph_boundary)
print(f"Found {len(enc_list)} ENCs intersecting with the graph boundary.")

ft_weights = weights_manager.get_feature_layers_from_classifier()

ft_weights


2025-10-07 22:34:02,954 - src.maritime_module.core.s57_data - INFO - Factory: Filtering ENCs by boundary...
2025-10-07 22:34:02,955 - src.maritime_module.core.s57_data - INFO - Factory: Getting ENC summary...
2025-10-07 22:34:03,022 - src.maritime_module.core.s57_data - INFO - Factory: Getting bounding boxes for 6369 ENCs...
Found 43 ENCs intersecting with the graph boundary.
2025-10-07 22:34:03,468 - src.maritime_module.core.graph - INFO - Generated 30 feature layer configs from classifier


{'canals': {'column': 'ft_hor_clearance',
  'attributes': ['horclr'],
  'aggregation': 'min'},
 'convyr': {'column': 'ft_ver_clearance',
  'attributes': ['verclr'],
  'aggregation': 'min'},
 'fairwy': {'column': 'ft_depth',
  'attributes': ['drval1'],
  'aggregation': 'min'},
 'rcrtcl': {'column': 'ft_depth',
  'attributes': ['drval1'],
  'aggregation': 'min'},
 'rectrc': {'column': 'ft_depth',
  'attributes': ['drval1'],
  'aggregation': 'min'},
 'dwrtcl': {'column': 'ft_depth',
  'attributes': ['drval1'],
  'aggregation': 'min'},
 'dwrtpt': {'column': 'ft_depth',
  'attributes': ['drval1'],
  'aggregation': 'min'},
 'depare': {'column': 'ft_depth',
  'attributes': ['drval1'],
  'aggregation': 'min'},
 'swpare': {'column': 'ft_depth',
  'attributes': ['drval1'],
  'aggregation': 'min'},
 'drgare': {'column': 'ft_depth',
  'attributes': ['drval1'],
  'aggregation': 'min'},
 'berths': {'column': 'ft_depth',
  'attributes': ['drval1'],
  'aggregation': 'min'},
 'docare': {'column': 'ft_h

In [6]:
# 3. Apply weights from static layers (e.g., land, fairways, obstructions)
summary = weights_manager.enrich_edges_with_features_postgis(enc_names= enc_list,
                                                               schema_name="graph",
                                                               graph_name=graph_name,
                                                               enc_schema="us_enc_all",
                                                               feature_layers=ft_weights
                                                             )



summary

2025-10-07 22:34:03,480 - src.maritime_module.core.graph - INFO - Generated 30 feature layer configs from classifier
2025-10-07 22:34:03,481 - src.maritime_module.core.graph - INFO - === PostGIS Feature Enrichment (Server-Side) ===
2025-10-07 22:34:03,481 - src.maritime_module.core.graph - INFO - Edges table: graph.fine_graph_01_opt_edges
2025-10-07 22:34:03,482 - src.maritime_module.core.graph - INFO - Layers schema: us_enc_all
2025-10-07 22:34:03,482 - src.maritime_module.core.graph - INFO - Processing 30 feature layers
2025-10-07 22:34:03,483 - src.maritime_module.core.graph - INFO - Initializing weight calculation columns
2025-10-07 22:34:03,494 - src.maritime_module.core.graph - INFO - Added column 'base_weight' to fine_graph_01_opt_edges
2025-10-07 22:34:03,500 - src.maritime_module.core.graph - INFO - Added column 'adjusted_weight' to fine_graph_01_opt_edges
2025-10-07 22:34:03,509 - src.maritime_module.core.graph - INFO - Added column 'blocking_factor' to fine_graph_01_opt_edge

{'ft_hor_clearance': 0,
 'ft_ver_clearance': 265,
 'ft_depth': 1417882,
 'ft_sounding': 1719,
 'ft_sounding_point': 1}

In [7]:

pg.get_table(table_name="fine_graph_01_opt_edges",
                     schema_name="graph")

2025-10-07 22:36:53,696 - src.maritime_module.utils.db_utils - INFO - Successfully connected to database 'ENC_db' for schema management.
2025-10-07 22:37:15,063 - src.maritime_module.utils.db_utils - INFO - Loaded 1416342 rows from 'graph.fine_graph_01_opt_edges'


,id,source_str,target_str,source_x,source_y,target_x,target_y,weight,geometry,base_weight,adjusted_weight,blocking_factor,penalty_factor,bonus_factor,ukc_meters,ft_hor_clearance,ft_ver_clearance,ft_depth,ft_sounding,ft_sounding_point
0,299666,"(-122.12533333333334, 37.50333333333333)","(-122.12200000000001, 37.5)",-122.125333,37.503333,-122.122000,37.500000,0.004714,"LINESTRING (-122.12533 37.50333, -122.122 37.5)",0.004714,0.004714,1.0,1.0,1.0,0.0,None,50.2,-2.4,NaN,NaN
1,1084239,"(-119.72200000000001, 34.28333333333333)","(-119.71866666666668, 34.28)",-119.722000,34.283333,-119.718667,34.280000,0.004714,"LINESTRING (-119.722 34.28333, -119.71867 34.28)",0.004714,0.004714,1.0,1.0,1.0,0.0,None,NaN,30.0,NaN,NaN
2,283369,"(-122.16533333333334, 37.5)","(-122.162, 37.50333333333333)",-122.165333,37.500000,-122.162000,37.503333,0.004714,"LINESTRING (-122.16533 37.5, -122.162 37.50333)",0.004714,0.004714,1.0,1.0,1.0,0.0,None,47.2,-2.4,NaN,NaN
3,305062,"(-122.11200000000001, 37.51)","(-122.11200000000001, 37.513333333333335)",-122.112000,37.510000,-122.112000,37.513333,0.003333,"LINESTRING (-122.112 37.51, -122.112 37.51333)",0.003333,0.003333,1.0,1.0,1.0,0.0,None,50.2,-2.4,NaN,NaN
4,297034,"(-122.132, 37.50333333333333)","(-122.12866666666667, 37.5)",-122.132000,37.503333,-122.128667,37.500000,0.004714,"LINESTRING (-122.132 37.50333, -122.12867 37.5)",0.004714,0.004714,1.0,1.0,1.0,0.0,None,50.2,-2.4,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1416337,409962,"(-121.85533333333335, 36.126666666666665)","(-121.852, 36.13)",-121.855333,36.126667,-121.852000,36.130000,0.004714,"LINESTRING (-121.85533 36.12667, -121.852 36.13)",0.004714,0.004714,1.0,1.0,1.0,0.0,None,NaN,182.8,NaN,NaN
1416338,480419,"(-121.662, 35.67666666666667)","(-121.662, 35.68)",-121.662000,35.676667,-121.662000,35.680000,0.003333,"LINESTRING (-121.662 35.67667, -121.662 35.68)",0.003333,0.003333,1.0,1.0,1.0,0.0,None,NaN,182.8,NaN,NaN
1416339,352253,"(-121.992, 36.01)","(-121.992, 36.013333333333335)",-121.992000,36.010000,-121.992000,36.013333,0.003333,"LINESTRING (-121.992 36.01, -121.992 36.01333)",0.003333,0.003333,1.0,1.0,1.0,0.0,None,NaN,914.4,NaN,NaN
1416340,525455,"(-121.552, 35.82333333333333)","(-121.54866666666668, 35.82333333333333)",-121.552000,35.823333,-121.548667,35.823333,0.003333,"LINESTRING (-121.552 35.82333, -121.54867 35.8...",0.003333,0.003333,1.0,1.0,1.0,0.0,None,NaN,182.8,NaN,NaN


In [8]:
config = weights_manager._load_config()

In [9]:
weights_manager.apply_static_weights_postgis(graph_name=graph_name,
                                             enc_names=enc_list,
                                             schema_name="graph",
                                             enc_schema="us_enc_all",
                                             static_layers=config["weight_settings"]["static_layers"],
                                             usage_bands=[3,4,5])

print("Static weights applied.")

2025-10-07 22:37:15,389 - src.maritime_module.core.graph - INFO - Filtered 43 ENCs to 40 based on usage bands [3, 4, 5]
2025-10-07 22:37:15,390 - src.maritime_module.core.graph - INFO - === PostGIS Static Weights Application (Three-Tier System) ===
2025-10-07 22:37:15,390 - src.maritime_module.core.graph - INFO - Edges table: graph.fine_graph_01_opt_edges
2025-10-07 22:37:15,390 - src.maritime_module.core.graph - INFO - Layers schema: us_enc_all
2025-10-07 22:37:15,391 - src.maritime_module.core.graph - INFO - Processing 15 layers
2025-10-07 22:37:15,391 - src.maritime_module.core.graph - INFO - Ensuring three-tier weight columns exist...
2025-10-07 22:37:15,397 - src.maritime_module.core.graph - INFO - Added 'wt_static_blocking' column to fine_graph_01_opt_edges
2025-10-07 22:37:15,401 - src.maritime_module.core.graph - INFO - Added 'wt_static_penalty' column to fine_graph_01_opt_edges
2025-10-07 22:37:15,403 - src.maritime_module.core.graph - INFO - Added 'wt_static_bonus' column to 

In [10]:
pg.get_table(table_name="fine_graph_01_opt_edges",
                     schema_name="graph")

2025-10-07 22:43:29,627 - src.maritime_module.utils.db_utils - INFO - Loaded 1416342 rows from 'graph.fine_graph_01_opt_edges'


,id,source_str,target_str,source_x,source_y,target_x,target_y,weight,geometry,base_weight,...,bonus_factor,ukc_meters,ft_hor_clearance,ft_ver_clearance,ft_depth,ft_sounding,ft_sounding_point,wt_static_blocking,wt_static_penalty,wt_static_bonus
0,58966,"(-122.73533333333334, 37.843333333333334)","(-122.73200000000001, 37.843333333333334)",-122.735333,37.843333,-122.732000,37.843333,0.003333,"LINESTRING (-122.73533 37.84333, -122.732 37.8...",0.003333,...,1.0,0.0,None,NaN,18.2,NaN,NaN,10.0,1.0,1.0
1,59053,"(-122.73533333333334, 37.916666666666664)","(-122.73200000000001, 37.916666666666664)",-122.735333,37.916667,-122.732000,37.916667,0.003333,"LINESTRING (-122.73533 37.91667, -122.732 37.9...",0.003333,...,1.0,0.0,None,NaN,-1.5,NaN,NaN,200.0,1.0,1.0
2,59051,"(-122.73533333333334, 37.916666666666664)","(-122.73200000000001, 37.913333333333334)",-122.735333,37.916667,-122.732000,37.913333,0.004714,"LINESTRING (-122.73533 37.91667, -122.732 37.9...",0.004714,...,1.0,0.0,None,NaN,-1.5,NaN,NaN,200.0,1.0,1.0
3,58997,"(-122.73533333333334, 37.87)","(-122.73200000000001, 37.86666666666667)",-122.735333,37.870000,-122.732000,37.866667,0.004714,"LINESTRING (-122.73533 37.87, -122.732 37.86667)",0.004714,...,1.0,0.0,None,NaN,0.0,NaN,NaN,10.0,1.0,1.0
4,59012,"(-122.73533333333334, 37.88333333333333)","(-122.73533333333334, 37.88666666666667)",-122.735333,37.883333,-122.735333,37.886667,0.003333,"LINESTRING (-122.73533 37.88333, -122.73533 37...",0.003333,...,1.0,0.0,None,NaN,0.0,NaN,NaN,10.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1416337,409962,"(-121.85533333333335, 36.126666666666665)","(-121.852, 36.13)",-121.855333,36.126667,-121.852000,36.130000,0.004714,"LINESTRING (-121.85533 36.12667, -121.852 36.13)",0.004714,...,1.0,0.0,None,NaN,182.8,NaN,NaN,10.0,1.0,1.0
1416338,480419,"(-121.662, 35.67666666666667)","(-121.662, 35.68)",-121.662000,35.676667,-121.662000,35.680000,0.003333,"LINESTRING (-121.662 35.67667, -121.662 35.68)",0.003333,...,1.0,0.0,None,NaN,182.8,NaN,NaN,10.0,1.0,1.0
1416339,352253,"(-121.992, 36.01)","(-121.992, 36.013333333333335)",-121.992000,36.010000,-121.992000,36.013333,0.003333,"LINESTRING (-121.992 36.01, -121.992 36.01333)",0.003333,...,1.0,0.0,None,NaN,914.4,NaN,NaN,10.0,1.0,1.0
1416340,525455,"(-121.552, 35.82333333333333)","(-121.54866666666668, 35.82333333333333)",-121.552000,35.823333,-121.548667,35.823333,0.003333,"LINESTRING (-121.552 35.82333, -121.54867 35.8...",0.003333,...,1.0,0.0,None,NaN,182.8,NaN,NaN,10.0,1.0,1.0


In [11]:
ply = PlotlyChart()
ply_fig = ply.create_base_map(mapbox_token="pk.eyJ1Ijoidmlrb250IiwiYSI6ImNtNHljdzQwbjBybWIybHNkZzhleXhqYXUifQ.Jz0cR4J6PdAnLutVBqj2WA")
ply.plotly_base_config(ply_fig)

{'displayModeBar': 'hover',
 'responsive': True,
 'scrollZoom': True,
 'displaylogo': False,
 'modeBarButtonsToRemove': ['zoomIn',
  'zoomOut',
  'pan',
  'select',
  'lassoSelect'],
 'modeBarButtonsToAdd': ['autoScale', 'hoverCompareCartesian']}

 ## Calculate Dynamic Weights

In [12]:

# Define vessel parameters for dynamic weight calculation
vessel_params = {
 'draft': 7.5,           # meters
 'height': 30.0,         # meters (for overhead clearance)
 'safety_margin': 2.0,   # meters (for under-keel clearance)
 'vessel_type': 'cargo'
}

# Define environmental conditions (optional)
env_conditions = {
 'weather_factor': 1.2,      # 1.0=good, >1.0=poor
 'visibility_factor': 1.1,   # 1.0=good, >1.0=poor
 'time_of_day': 'day'        # 'day' or 'night'
}

# Calculate the final weights
# This function uses the three-tier system: Blocking, Penalties, and Bonuses
weights_manager.calculate_dynamic_weights_postgis(
 graph_name=graph_name,
 schema_name="graph",
 vessel_parameters=vessel_params,
 environmental_conditions=env_conditions,
)

print("Dynamic weights calculated.")

2025-10-07 22:43:30,583 - src.maritime_module.core.graph - INFO - === Dynamic Weight Calculation (PostGIS - Three-Tier System) ===
2025-10-07 22:43:30,584 - src.maritime_module.core.graph - INFO - Vessel: type=cargo, draft=7.5m, height=30.0m
2025-10-07 22:43:30,585 - src.maritime_module.core.graph - INFO - Safety margin: 2.0m → 2.64m (adjusted)
2025-10-07 22:43:30,585 - src.maritime_module.core.graph - INFO - Environment: weather=1.2, visibility=1.1, time=day
2025-10-07 22:43:30,585 - src.maritime_module.core.graph - INFO - Max penalty cap: 50.0
2025-10-07 22:43:55,961 - src.maritime_module.core.graph - INFO - Tier 1: Calculating blocking factors...
2025-10-07 22:44:13,592 - src.maritime_module.core.graph - INFO - Tier 2: Calculating penalty factors...
2025-10-07 22:44:24,845 - src.maritime_module.core.graph - INFO - Tier 3: Calculating bonus factors...
2025-10-07 22:44:48,909 - src.maritime_module.core.graph - INFO - Calculating adjusted weights...
2025-10-07 22:45:12,931 - src.mariti

## Inspect Weighted Graph

In [13]:
pg.get_table(table_name="fine_graph_01_opt_edges",
             schema_name="graph")

2025-10-07 22:45:36,275 - src.maritime_module.utils.db_utils - INFO - Loaded 1416342 rows from 'graph.fine_graph_01_opt_edges'


,id,source_str,target_str,source_x,source_y,target_x,target_y,weight,geometry,base_weight,...,bonus_factor,ukc_meters,ft_hor_clearance,ft_ver_clearance,ft_depth,ft_sounding,ft_sounding_point,wt_static_blocking,wt_static_penalty,wt_static_bonus
0,44668,"(-122.77533333333335, 37.64666666666667)","(-122.772, 37.65)",-122.775333,37.646667,-122.772000,37.650000,0.004714,"LINESTRING (-122.77533 37.64667, -122.772 37.65)",0.004714,...,0.9,NaN,None,NaN,18.2,NaN,NaN,10.0,1.0,1.0
1,1084239,"(-119.72200000000001, 34.28333333333333)","(-119.71866666666668, 34.28)",-119.722000,34.283333,-119.718667,34.280000,0.004714,"LINESTRING (-119.722 34.28333, -119.71867 34.28)",0.004714,...,0.9,NaN,None,NaN,30.0,NaN,NaN,1.0,1.0,1.0
2,171705,"(-122.43533333333335, 36.653333333333336)","(-122.43200000000002, 36.656666666666666)",-122.435333,36.653333,-122.432000,36.656667,0.004714,"LINESTRING (-122.43533 36.65333, -122.432 36.6...",0.004714,...,0.9,NaN,None,NaN,1828.8,NaN,NaN,10.0,1.0,1.0
3,108521,"(-122.602, 37.50333333333333)","(-122.59866666666667, 37.5)",-122.602000,37.503333,-122.598667,37.500000,0.004714,"LINESTRING (-122.602 37.50333, -122.59867 37.5)",0.004714,...,0.9,NaN,None,NaN,18.2,NaN,NaN,10.0,1.0,1.0
4,125388,"(-122.55866666666668, 37.346666666666664)","(-122.55866666666668, 37.35)",-122.558667,37.346667,-122.558667,37.350000,0.003333,"LINESTRING (-122.55867 37.34667, -122.55867 37...",0.003333,...,0.9,NaN,None,NaN,30.0,NaN,NaN,10.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1416337,859143,"(-120.712, 35.07)","(-120.70866666666667, 35.07)",-120.712000,35.070000,-120.708667,35.070000,0.003333,"LINESTRING (-120.712 35.07, -120.70867 35.07)",0.003333,...,1.0,-7.5,None,NaN,0.0,NaN,NaN,10.0,1.0,1.0
1416338,877723,"(-120.662, 34.57)","(-120.65866666666668, 34.57)",-120.662000,34.570000,-120.658667,34.570000,0.003333,"LINESTRING (-120.662 34.57, -120.65867 34.57)",0.003333,...,1.0,-7.5,None,NaN,0.0,NaN,NaN,10.0,1.0,1.0
1416339,857521,"(-120.71533333333335, 34.8)","(-120.712, 34.8)",-120.715333,34.800000,-120.712000,34.800000,0.003333,"LINESTRING (-120.71533 34.8, -120.712 34.8)",0.003333,...,1.0,-7.5,None,NaN,0.0,NaN,NaN,10.0,1.0,1.0
1416340,887974,"(-120.632, 34.82)","(-120.62866666666667, 34.82)",-120.632000,34.820000,-120.628667,34.820000,0.003333,"LINESTRING (-120.632 34.82, -120.62867 34.82)",0.003333,...,1.0,-7.5,None,NaN,0.0,NaN,NaN,10.0,1.0,1.0


 ## Save the Weighted Graph

In [14]:

# Save the graph with all its new weight attributes to a new GeoPackage file
output_path = output_dir / "weighted_graph_example.gpkg"
weights_manager.save_weighted_graph_to_gpkg(
     graph=G_final_weighted,
     output_path=str(output_path),
     include_metadata=True
    )
print(f"Weighted graph saved to: {output_path}")

NameError: name 'G_final_weighted' is not defined

In [ ]:
if G:
     # Save the graph with all its new weight attributes to a new GeoPackage file
     weights_manager.save_weighted_graph_to_postgis(
         graph=G_final_weighted,
         table_prefix="graph_weigted",
         schema="graph",
         drop_existing=True
     )
